## An implementation of the F-adjoint propagation from scratch

In [8]:
import numpy as np

# The activation function and it's derivative:
sigma = lambda x: 1/(1+np.exp(-x))
d_sigma= lambda x: sigma(x)*(1-sigma(x))
# The F-propagation of the data X through the weight W:
def F_prop(X, W):
    L=len(W)
    F_X = {'X0':X.T}
    for h in range(1, L+1):
        F_X[f'Y{h}']=np.dot(W[f'W{h}'], F_X[f'X{h-1}'])
        if h!=L:
            F_X[f'X{h}']=np.concatenate((sigma(F_X[f'Y{h}']), np.ones((1,X.shape[0]))), axis = 0)
        else:
            F_X[f'X{h}']=sigma(F_X[f'Y{h}'])
    return F_X
# The F-adjoint propagation of the data (X,y) through the weight W:
def F_star_prop(X,Y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star={f'X*{L}':(F_X[f'X{L}']-Y)}
    Y_star={}
    for h in reversed(range(1, L+1)):
        FX_star[f'Y*{h}']=FX_star[f'X*{h}']*(d_sigma(F_X[f'Y{h}']))
        FX_star[f'X*{h-1}']=np.delete(W[f'W{h}'], -1, axis = 1).T.dot(FX_star[f'Y*{h}'])
    return FX_star
# The gradient update, with respect to the weight W, of the data (X,y):
def Grad_star(X,y, W, eta):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star=F_star_prop(X,y, W)
    Grad={}
    for h in range(1, L+1):
        Grad[f'W{h}']=W[f'W{h}']-eta*FX_star[f'Y*{h}'].dot(F_X[f'X{h-1}'].T)
    return Grad

In [10]:
# Learning the XOR dataset
X = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
y = np.array([0, 1, 1, 0])
# Neural network with the architecture A[3,2,1]
W1=np.array([[1, 0, 0], [0, 1, 0]])
W2=np.array([[0, 0, 1]])
W={'W1':W1, 'W2':W2}
##
X2=F_prop(X, W)['X2']
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
   # print("================================")
    print(W, X2, X2_c,k)
    print("================================================================================================")
    W=Grad_star(X,y, W, 4.3)
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    k+=1

{'W1': array([[1, 0, 0],
       [0, 1, 0]]), 'W2': array([[0, 0, 1]])} [[0.731 0.731 0.731 0.731]] [[1 1 1 1]] 0
{'W1': array([[1., 0., 0.],
       [0., 1., 0.]]), 'W2': array([[-0.481, -0.481,  0.219]])} [[0.435 0.408 0.408 0.381]] [[0 0 0 0]] 1
{'W1': array([[ 0.978, -0.037, -0.04 ],
       [-0.037,  0.978, -0.04 ]]), 'W2': array([[-0.236, -0.236,  0.603]])} [[0.592 0.579 0.579 0.566]] [[1 1 1 1]] 2
{'W1': array([[ 0.986, -0.034, -0.022],
       [-0.034,  0.986, -0.022]]), 'W2': array([[-0.434, -0.434,  0.272]])} [[0.461 0.437 0.437 0.413]] [[0 0 0 0]] 3
{'W1': array([[ 0.973, -0.061, -0.047],
       [-0.061,  0.973, -0.047]]), 'W2': array([[-0.266, -0.266,  0.54 ]])} [[0.57  0.556 0.556 0.541]] [[1 1 1 1]] 4
{'W1': array([[ 0.979, -0.06 , -0.032],
       [-0.06 ,  0.979, -0.032]]), 'W2': array([[-0.405, -0.405,  0.305]])} [[0.477 0.455 0.455 0.433]] [[0 0 0 0]] 5
{'W1': array([[ 0.97 , -0.081, -0.048],
       [-0.081,  0.97 , -0.048]]), 'W2': array([[-0.284, -0.284,  0.498]])} [[0.5

# Code to calculate the $F$ and $F_*$ propagations

In [15]:
import numpy as np
import pandas as pd
#
sigma = lambda x: 1/(1+np.exp(-x))
d_sigma= lambda x: sigma(x)*(1-sigma(x))
#
def F_prop(X, W):
    L=len(W)
    Y_h = {}
    F_X = {'X0':X.T}
    for h in range(1, L+1):
        F_X[f'Y{h}'] = np.round(np.dot(W[f'W{h}'], F_X[f'X{h-1}']),2)
        if h!=L:
            F_X[f'X{h}'] = np.round(np.concatenate((sigma(F_X[f'Y{h}']), np.ones((1,1))), axis = 0),2)
        else:
            F_X[f'X{h}'] = np.round(sigma(F_X[f'Y{h}']),2)
    return F_X
def F_star_prop(X,y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star={f'X*{L}':np.round((F_X[f'X{L}']-y),2)}
    Y_star={}
    for h in reversed(range(1, L+1)):
        FX_star[f'Y*{h}'] =np.round(FX_star[f'X*{h}']*(d_sigma(F_X[f'Y{h}'])),2)
        FX_star[f'X*{h-1}']=np.round(np.delete(W[f'W{h}'], -1, axis = 1).T.dot(FX_star[f'Y*{h}']),2)
    return FX_star

W1=np.array([[1,-1,1],[-1,1,-1]])
W2=np.array([[1,-1,1]])
W={"W1":W1,"W2":W2}
X= np.array([[0,0,1]])
Y=np.array([[1]])
F_X = F_prop(X, W)
FX_star=F_star_prop(X,Y, W)
df1=pd.DataFrame([F_X])
df2=pd.DataFrame([FX_star])

In [16]:
F_X

{'X0': array([[0],
        [0],
        [1]]),
 'Y1': array([[ 1],
        [-1]]),
 'X1': array([[0.73],
        [0.27],
        [1.  ]]),
 'Y2': array([[1.46]]),
 'X2': array([[0.81]])}

In [17]:
FX_star

{'X*2': array([[-0.19]]),
 'Y*2': array([[-0.03]]),
 'X*1': array([[-0.03],
        [ 0.03]]),
 'Y*1': array([[-0.01],
        [ 0.01]]),
 'X*0': array([[-0.02],
        [ 0.02]])}

In [18]:
df1 #Valurs correctes: X1=(0.73,0.27,1); Y2=1.46 et X2=0.81

,X0,Y1,X1,Y2,X2
0,"[[0], [0], [1]]","[[1], [-1]]","[[0.73], [0.27], [1.0]]",[[1.46]],[[0.81]]


In [19]:
df2#Valurs correctes: X2_*=-0.19; Y2_*=-0.03 et X1_*=[[-0.03], [0.03]] Y1_*=[[-0.01], [0.01]],X0_*[[-0.02], [0.02]]

,X*2,Y*2,X*1,Y*1,X*0
0,[[-0.19]],[[-0.03]],"[[-0.03], [0.03]]","[[-0.01], [0.01]]","[[-0.02], [0.02]]"
